In [1]:
import requests as req
import json
import io
# import time
# import os
# import random

with open('config.json', 'r') as read_file:
    client = json.load(read_file)

In [16]:
# functions
def ceildiv(a, b):
    return -(-a // b)

def id_generator(size: int = 6):
    chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
    return ''.join(random.choice(chars) for _ in range(size))

def scp_query(query: str = '', inst: str = '', city: str = '', country: str = '', open_access: str = '',
    start_year: int = 0, end_year: int = 0):
    
    inst_q, city_q, country_q = '', '', ''
    if inst:
        inst_q = ' AND '.join(inst.lower().split(' '))
    if city:
        city_q = ' AND '.join(city.lower().split(' '))
    if country:
        country_q = ' AND '.join(country.lower().split(' '))
    query = [
        (f'{query}' if query else ''),
        (f'AFFIL({inst_q})' if inst_q else ''),
        (f'AFFILCITY({city_q})' if city_q else ''),
        (f'AFFILCOUNTRY({country_q})' if country_q else ''),
        (f'ACCESSTYPE({open_access})' if open_access else ''),
        (f'(PUBYEAR > {start_year - 1})' if start_year else ''),
        (f'(PUBYEAR < {end_year + 1})' if end_year != 0 else ''),
    ]
    query = filter(lambda element: element, query)
    return ' AND '.join(query)

def scp_lookup(client:dict, query: str):
    base = 'https://api.elsevier.com/metrics/institution/search'
    params = {
        'query': query,
        'apiKey': client['apikey'],
        'insttoken': client['insttoken'],
#         'view': 'COMPLETE',
#         'start': f'{start_cursor + (cnt * count_per_page)}',
#         'count': f'{count_per_page}',
    }
    headers = {'Accept': 'application/json'}
    print(query)
    print(params)
    return req.get(url=base, params=params)

def export_file(data: str, inst: str, year: int,
    open_access: str = '', file_id: str = '', folder: str = '', cnt: int = 0, extension: str = '.txt'):
    
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    cnt = str(cnt).zfill(3)
    isnt = inst.replace(' ', '-')
    if extension[0] != '.':
        extension += '.'
    file_name = f'{inst}_y{year}_{cnt}'
    if open_access:
        file_name += f'_access-{open_access.upper()}'
    if file_id:
        file_name += f'_{file_id}'
    file_name += f'_{int(time.time())}{extension}'
    file_name = os.path.join(folder, file_name)
    with io.open(file_name, 'w', encoding='utf8') as f:
        f.write(data) 

def scp_papers(client: dict, query: str, years: list, inst: str, folder: str = ''):
    file_id = id_generator()
    for year in range(years[0], years[1]):
        q = scp_query(query=query, start_year=year, end_year=year)
        res = scp_lookup(client=client, query=q, cnt=0)
        total_results = int(res.json()['search-results']['opensearch:totalResults'])
        
        export_file(data=res.text, file_id=file_id, inst=inst, year=year, folder=folder, cnt=0, extension='.txt')
        if total_results > 5000:
            print(f'WARNING: paper count: {total_results}. Retrieving only the first 5000')
        page_count = min(ceildiv(total_results, 25), 200) # 25 is the max number of papers in each request
        for cnt in range(1, page_count):
            res = scp_lookup(client=client, query=q, cnt=cnt)
            export_file(data=res.text, file_id=file_id, inst=inst, year=year, folder=folder, cnt=cnt, extension='.txt')

In [17]:
res = scp_lookup(client=client, query='name(Harvard) or name(Massachusetts Institute of Technology)')

name(Harvard) or name(Massachusetts Institute of Technology)
{'query': 'name(Harvard) or name(Massachusetts Institute of Technology)', 'apiKey': 'ef153b4cfcdff96a0d11d90c5f231676', 'insttoken': 'fa135528e1510f441eead9b048db4aa0'}


In [13]:
res.json()

{'service-error': {'status': {'statusCode': 'AUTHENTICATION_ERROR',
   'statusText': 'Invalid API Key: valid apikey credentials required.'}}}